<a href="https://colab.research.google.com/github/itaborai83/ecd221-ML-trabalho/blob/main/ML_7_Projeto_completo_de_Classifica%C3%A7%C3%A3o_Bin%C3%A1ria_Telco_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Especialização em Ciência de Dados - PUC-Rio
# Machine Learning - Prof. Tatiana Escovedo
# Projeto completo de Classificação Binária


## 1. Definição do Problema

https://www.kaggle.com/datasets/blastchar/telco-customer-churn

**Informações sobre os atributos:**

Adicionar

In [ ]:
# função de correlação que funciona com variáveis categóricas e numéricas
! pip install phik mlxtend

# https://stackoverflow.com/questions/61867945/python-import-error-cannot-import-name-six-from-sklearn-externals
! pip install six 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 690 kB 14.5 MB/s 


In [ ]:
# Imports
import sys
import math
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms # para tratamento de missings
from matplotlib import cm
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, plot_roc_curve
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer # transformador de colunas, usado para tratamento das variáveis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# a versão do Stacking classifier disponível no Colab não aceita classificadores pré-treinados
# from sklearn.ensemble import StackingClassifier
# correção de bug na biblioteca que importa uma dependência de maneira indireta
# https://stackoverflow.com/questions/61867945/python-import-error-cannot-import-name-six-from-sklearn-externals
import six
sys.modules['sklearn.externals.six'] = six
from mlxtend.classifier import EnsembleVoteClassifier

import phik
from phik.report import plot_correlation_matrix
from phik import report

# redução de dimensionalidade
from sklearn.decomposition import PCA

# para baixar os modelos já treinados
import urllib.request

from pprint import pprint, pformat

import pickle # para persistência de modelos

In [ ]:
# setup ambiente

# supressão de warnings
import warnings
warnings.filterwarnings("ignore")

# configura pandas para exibição de apenas duas casas decimais nas variaveis
pd.set_option('display.float_format', lambda x: '%0.2f' % x)

# configura cores do Seaborn
sns.set()

In [ ]:
# constantes

DATA_URL                    = f"https://raw.githubusercontent.com/itaborai83/ecd221-ML-trabalho/main/telco-churn.csv"
FIELD_SEPARATOR             = ","
IMPORT_COLUMN_NAMES         = [
    "customer_id"
,   "gender"
,   "senior_citizen"
,   "partner"
,   "dependents"
,   "tenure"
,   "phone_service"
,   "multiple_lines"
,   "internet_service"
,   "online_security"
,   "online_backup"
,   "device_protection"
,   "tech_support"
,   "streaming_tv"
,   "streaming_movies"
,   "contract"
,   "paperless_billing"
,   "payment_method"
,   "monthly_charges"
,   "total_charges"
,   "churn"
]
COLUMN_NAMES = [
    "gender"
,   "senior_citizen"
,   "partner"
,   "dependents"
,   "phone_service"
,   "multiple_lines"
,   "internet_service"
,   "online_security"
,   "online_backup"
,   "device_protection"
,   "tech_support"
,   "streaming_tv"
,   "streaming_movies"
,   "contract"
,   "paperless_billing"
,   "payment_method"
,   "tenure"
,   "monthly_charges"
,   "total_charges"
,   "churn"
]
TARGET_VARIABLE = "churn"
BOOLEAN_FEATURES = [
    "senior_citizen"
,   "partner"
,   "dependents"
,   "phone_service"
,   "paperless_billing"
]
CATEGORICAL_FEATURES = [
    "multiple_lines"
,   "internet_service"
,   "online_security"
,   "online_backup"
,   "device_protection"
,   "tech_support"
,   "streaming_tv"
,   "streaming_movies"
,   "contract"
,   "payment_method"
]
NUMERICAL_FEATURES                  = ["tenure", "monthly_charges", "total_charges"]
NUMERICAL_FEATURES_AFTER_FEAT_ENG   = ["tenure", "monthly_charges", "total_charges", "client_factor", "internet_factor", "financial_factor", "multi_factor"]
BOOLEAN_MAP                         = {"No": 0, "Yes": 1}
TEST_PCT_SIZE                       = 0.3 # 30% do conjunto de dados
RANDOM_STATE                        = 42
SCORING_METRIC                      = "roc_auc"
K_FOLDS                             = 6
RETURN_TRAIN_SCORE                  = False
OUTPUT_TRAINING_FILE_TMPLT          = "RandomizedSearchCV_{algo}.xlsx"
QUICK_RUN                           = False
DOWNLOAD_MODELS_URL                 = "https://github.com/itaborai83/ecd221-ML-trabalho/raw/main/"
MODEL_SETTINGS                      = {
     "logreg"           : { "model_file": "logreg.pkl", "train": True, "download": False }
,    "knn"              : { "model_file": "knn.pkl",    "train": True, "download": False }
,    "nb"               : { "model_file": "nb.pkl",     "train": True, "download": False }
,    "dt"               : { "model_file": "dt.pkl",     "train": True, "download": False }
,    "svm"              : { "model_file": "svm.pkl",    "train": True, "download": False }
,    "ada"              : { "model_file": "ada.pkl",    "train": True, "download": False }
,    "gb"               : { "model_file": "gb.pkl",     "train": True, "download": False }
,    "rf"               : { "model_file": "rf.pkl",     "train": True, "download": False }
,    "vt"               : { "model_file": "vt.pkl",     "train": True, "download": False }
}

## 2. Carga de Dados

Iremos usar o pacote Pandas ( Python Data Analysis Library) para carregar de um arquivo .csv sem cabeçalho disponível online.

Com o dataset carregado, iremos explorá-lo um pouco.

In [ ]:
# Carrega arquivo csv usando Pandas usando uma URL
churn_df = pd.read_csv(
    DATA_URL
,   names     = IMPORT_COLUMN_NAMES
,   skiprows  = 1
,   delimiter = ','
)

# transforma a variável target em uma variável numérica
churn_df["churn"] = churn_df["churn"].map(BOOLEAN_MAP)

# excluindo a variável customer_id
del churn_df["customer_id"]

for algo in MODEL_SETTINGS:
  model_setting = MODEL_SETTINGS[algo]
  download      = model_setting["download"]
  model_file    = model_setting["model_file"]
  if download:
    url = DOWNLOAD_MODELS_URL + model_file
    urllib.request.urlretrieve(url, model_file)

In [ ]:
churn_df.head()

## 3. Análise de Dados

### 3.1. Estatísticas Descritivas

Vamos iniciar examinando as dimensões do dataset, suas informações e alguns exemplos de linhas.

In [ ]:
# Mostra as dimensões do dataset
print(churn_df.shape)

In [ ]:
# Mostra as informações do dataset
print(churn_df.info())

o dataset possui predominantesmente dados categóricos, representados acima como objetos com tipo de dados **object**

In [ ]:
# Mostra as 10 primeiras linhas do dataset
churn_df.head(10)

A variável **monthly_charges** aparenta ser numérica.


In [ ]:
# a conversão do tipo da coluna abaixo falha com o erro "ValueError: could not convert string to float"
# churn_df["total_charges"] = churn_df["total_charges"].astype(float)

# tenta identificar valores problemáticos
def convertible_to_float(value):
  try:
    f = float(value)
    return True
  except:
    return False
is_float = churn_df["total_charges"].map(convertible_to_float)
churn_df["total_charges_is_float"] = is_float
display(churn_df[ churn_df["total_charges_is_float"] == False ][ "total_charges" ].map(repr))


O dataframe possui 11 registros com valor ' ' na coluna total_charges. Os valores serã convertidos para zero e posteriormente analisados


In [ ]:
def convert_total_charges(value):
    return 0.0 if value == ' ' else value

churn_df["total_charges"] = churn_df["total_charges"].map(convert_total_charges).astype(float)
del churn_df["total_charges_is_float"]

In [ ]:
# Mostra as 10 últimas linhas do dataset
churn_df.tail(10)

Diferente das outras variáveis categóricas, a variável **senior_citizem** está utilizando 0's e 1's para representar verdadeiro e falso. Iremos converter estes dados momentamenteamente por questões de consistência

In [ ]:
churn_df["senior_citizen"] = churn_df["senior_citizen"].map({1: "Yes", 0: "No"})

É sempre importante verificar o tipo do atributos do dataset, pois pode ser necessário realizar conversões. Já fizemos anteriormente com o comando info, mas vamos ver uma outra forma de verificar a natureza de cada atributo e então exibir um resumo estatístico do dataset.

In [ ]:
# Verifica o tipo de dataset de cada atributo
churn_df.info()

In [ ]:
# Faz um resumo estatístico das variáveis numéricas do dataset (média, desvio padrão, mínimo, máximo e os quartis)
churn_df.describe()

a variável **total_charges** é numericamente muito próxima das variáveis **tenure** e **monthly_charges** multiplicadas e por isso será removida

In [ ]:
churn_df["tenure*monthly"] = churn_df["tenure"] * churn_df["monthly_charges"]
churn_df["churn"] = churn_df.pop("churn")
display(churn_df.describe())
del churn_df["tenure*monthly"]

O dataset não possui valores faltantes ou valores anormalmente altos ou baixos em suas variaveis numéricas

Vamos agora verificar se o dataset tem as classes balanceadas para que possamos tratar o desbalanceamento posteriormente, se necessário. Veremos que as classes 0 (permaneceu cliente) e 1 (não permaneceu cliente) estão desbalanceadas. Vamos guardar esta informação, pois possivelmente precisaremos realizar algum tipo de tratamento nas próximas etapas.

In [ ]:
# distribuição das classes
pd.crosstab(churn_df[TARGET_VARIABLE], churn_df[TARGET_VARIABLE], normalize=True)

Não está claro se o balanceamento de classes e a distribuição das variáveis do dataset correspondem à dados observados na vida real ou se algum tipo de curadoria ou rebalanceamento foi realizado (dado que a rotatividade de clientes na ordem de 27% parece ser um percentual muito elevado)

In [ ]:
pd.crosstab(churn_df["gender"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

A variável **gender** está balanceada no dataset e também balanceado quando comparada à variável target. Não está claro se essa variável é realmente útil.

In [ ]:
pd.crosstab(churn_df["senior_citizen"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

A variável categórica **senior_citizen** parece influenciar a rotatividade de clientes.

Se por um lado 84% dos clientes não estão na terceira idade, aproximadamente 75% (20%/27%) dos ex-clientes pertencem à esta categoria.

Por sua vez, os clientes pertencentes à terceira idade correspondem a apenas 16% dos clientes, mas desses 16%, 43% deixaram de ser clientes.

Em outras palavras. Em termos absolutos, um ex-cliente tem maior probabilidade de não pertencer a terceira idade. Em contra-partida, dado que um cliente pertence à terceira idade, ele tem maior probabilidade de se tornar um ex-cliente.




In [ ]:
pd.crosstab(churn_df["partner"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

A variável **partner**, que indica se um cliente possui ou não algum tipo de parceiro/cônjuge parece estar bem distribuída no conjunto de dados com aproximadamente metade dos clientes tendo algum parceiro e a outra metade não.

Entretanto, podemos que observar que a retenção dos clientes tende a ser pior entre os clientes que não possuem algum tipo de parceiro, pois aproximadamente 2/3 dos ex-clientes enquadram-se nessa situação

In [ ]:
pd.crosstab(churn_df["dependents"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

A variável **dependents**, que indica se um cliente possui ou não algum tipo de dependente parece ser um forte indicador sobre a rotatividade dos clientes pois mais de 81% (22%/27%) dos ex-clientes não possuem dependentes.

In [ ]:
g = sns.FacetGrid(churn_df, col="churn", size=3)
g.map_dataframe(plt.hist, x="tenure", alpha=0.5)
g.add_legend();

A variável **tenure**, que representa o número de meses em que um cliente possui algum tipo de relacionamento comercial com a empresa, parece indicar que a empresa oferece planos/contratos de duração máxima de 72 meses. 

Analisando os clientes que permaneceram com a empresa, podemos obseravar uma distribuição em forma de U, onde um número sigificativo de clientes possuem poucos meses de relacionamento e um número ainda maior parece estar se aproximando do teto de 72 meses.

Por sua vez, a análise dos futuros ex-clientes indicam que a maioria parece cancelar seus serviços nos primeiros meses de relacionamento com a empresa e, passado o tempo, fidelizam-se.

Estas distribuições parecem indicar a necessidade de que medidas de fidelização sejam tomadas nos críticos primeiros meses.

In [ ]:
pd.crosstab(churn_df["phone_service"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

A variável **phone_service** está extremamente desbalanceada e o fato de um usuário não possuir nenhum tipo de serviço telefônico contratado parece não ser expressivo em termos absolutos ou relativos

In [ ]:
pd.crosstab(churn_df["multiple_lines"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

A variável **multiple_lines** parece não informações significativas para determinar a rotatividade dos clientes

In [ ]:
pd.crosstab(churn_df["internet_service"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

A variável **internet_service** usada para identificar qual o tipo de fornecimento de internet o usuário possui (ou não) indica que quase 2/3 entre todos os ex-clientes parecem ter contratado internet por fibra ótica.

Isso parece indicar a existência de problemas na qualidade do fornecimento do serviço ou na precificação do mesmo.

In [ ]:
g = sns.FacetGrid(churn_df, col="internet_service", row="churn", size=3)
g.map_dataframe(plt.hist, x="monthly_charges", alpha=0.5)
g.add_legend();

O serviço de fibra ótica, conforme esperado, parece contribuir com o aumento dos custos mensais dos clientes e consequentemente afetando negativamente na retenção dos clientes que contrataram esse serviço.

In [ ]:
pd.crosstab(churn_df["online_security"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

In [ ]:
pd.crosstab(churn_df["online_backup"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

In [ ]:
pd.crosstab(churn_df["device_protection"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

In [ ]:
pd.crosstab(churn_df["tech_support"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

As variáveis **online_security**, **online_backup**, **device_protection** e **tech_support** comportam-se de maneira parecida. Em todos os casos, a não contratação dos serviços parecem aumentar a probabilidade do cliente desfazer seu relacionamento com a empresa.

Em outras palavras, a contratação desses serviços parece ser indicativo de que um cliente está fidelizado.


In [ ]:
pd.crosstab(churn_df["streaming_tv"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

In [ ]:
pd.crosstab(churn_df["streaming_movies"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

As variáveis **streaming_movies** e **streaming_tv** parecem não influenciar de forma significativa a rotatividade dos clientes, possivelmente por se tratarem de serviços de uso muito prevalente.

In [ ]:
pd.crosstab(churn_df["contract"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

Aproximadamente 85% da rotatividade dos clientes ocorrem entre aqueles que possuem contratos mensais, aparentando-ser um dos principais fatores que contribuem com a rotatividade dos clientes.

Os clientes tendem a permanecer fidelizados pela duração do contrato vigente que assinaram com a empresa de telefonia/telecomunicações.

In [ ]:
pd.crosstab(churn_df["paperless_billing"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

Clientes que possuem cobrança digital tendem a ter uma rotatividade mais alta do que clientes cobrados de maneira impressa.

In [ ]:
pd.crosstab(churn_df["payment_method"], churn_df[TARGET_VARIABLE], normalize=True, margins=True)

os métodos de pagamento presentes no dataset estão todos balanceados quando compara-se os cliente que permaneceram contratando os serviços da empresa.

Entre os ex-clientes, a probabilidade de que o método de pagamento empregado tenha sido cheques eletrônicos (depósitos automáticos) é consideravelmente maior, conforme dados acima.

In [ ]:
g = sns.FacetGrid(churn_df, hue="churn", size=3)
g.map_dataframe(plt.hist, x="monthly_charges", alpha=0.5)
g.add_legend();

Conforme analisada realizada anteriormente, a variável **monthly_charges** parece ter uma significativa correlação positiva com a rotatividade dos clientes. 

Foi observado que o serviço contratado de internet de fibra ótica parece ser, dentre os serviços existentes, aquele que mais contribui com o aumento do valor mensal cobrado e que, consequentemente, mais influencia na rotatividade.

Ademais, os dados dos ex-clientes parecem indicar a existência de uma distribuição tri-modal composta primeiramente pelos clientes que utilizam-se apenas dos serviços de telefonia, subsequentemente, os clientes que utilizam-se da internet via DSL e, por último, os clientes que se utilizam da internet via fibra ótica.

Com base nas análises acima, as seguintes ações serão tomadas

*   Variável **gender** será eliminada;
*   Variável **total_charges** será elimnada;
*   As variáveis categoricas serão posteriormente tratadas



In [ ]:
del churn_df["gender"]
churn_df["total_charges"] = churn_df["monthly_charges"] * churn_df["tenure"]
churn_df["churn"] = churn_df.pop("churn")


### 3.2. Visualizações Unimodais Dados Numéricos

Vamos criar agora um histograma para cada atributo do dataset. Veremos que os atributos age, pedi e test seguem uma distribuição exponencial, e que as colunas mass e press seguem uma distribuição aproximadamente normal.

In [ ]:
# Histograma
churn_df.hist(figsize = (15,15))
plt.show()

O Gráfico de Densidade, ou Density Plot, é bem parecido com o histograma, mas com uma visualização um pouco diferente. Com ele, pode ser mais fácil identificar a distribuição do atributos do dataset. Assim como fizemos com o histograma, vamos criar um density plot para cada atributo do dataset.

Veremos que muitos dos atributos têm uma distribuição distorcida. Uma transformação como a Box-Cox, que pode aproximar a distribuição de uma Normal, pode ser útil neste caso.

In [ ]:
# Density Plot
churn_df.plot(kind = 'density', subplots = True, sharex = False, figsize = (15,10))
plt.show()

Vamos agora trabalhar com boxplots. No **boxblot**, a linha no centro (vermelha) representa o valor da mediana (segundo quartil ou p50). A linha abaixo é o 1o quartil (p25) e a linha acima o terceiro quartil (p75). O boxplot ajuda a ter uma ideia da dispersão dos dataset e os possíveis outliers.

*OBS: Se um ponto do dataset é muito distante da média (acima de 3 desvios padrão da média), pode ser considerado outlier.*

Nos gráficos bloxplot, veremos que a dispersão dos atributos do dataset é bem diferente.

In [ ]:
# Boxplot
churn_df.plot(kind = 'box', subplots = True, layout = (3,3), sharex = False, sharey = False, figsize = (15,10))
plt.show()

### 3.3. Visualizações Multimodais

Ao visualizar as correlações entre os atributos através da matriz de correlação, perceberemos que parece haver alguma estrutura na ordem dos atributos. O azul ao redor da diagonal sugere que os atributos que estão próximos um do outro são geralmente mais correlacionados entre si. Os vermelhos também sugerem alguma correlação negativa moderada, a medida que os atributos 

Vamos agora verificar a covariância entre as variáveis numéricas do dataset. A **covariância** representa como duas variáveis numéricas estão relacionadas. Existem várias formas de calcular a correlação entre duas variáveis, como por exemplo, o coeficiente de correlação de Pearson, que pode ser:
* Próximo de -1 : há uma correlação negativa entre as variáveis, 
* Próximo de +1: há uma correlação positiva entre as variáveis. 
* 0: não há correlação entre as variáveis.

<i>OBS: Esta informação é relevante porque alguns algoritmos como regressão linear e regressão logística podem apresentar problemas de performance se houver atributos altamente correlacionados. Vale a pena consultar a documentação do algoritmo para verificar se algum tipo de tratamento de dataset é necessário.</i>

Falamos anteriormente da importância da correlação entre os atributos, e agora iremos visualizar esta informação em formato gráfico. A **matriz de correlação** exibe graficamente a correlação entre os atributos numéricos do dataset.estão mais distantes um do outro na ordenação. 

O código a seguir exibe a matriz de correlação.

In [ ]:
# Matriz de Correlação com Matplotlib Seaborn
interval_cols = ['tenure', 'monthly_charges', 'total_charges']

phik_overview_df = churn_df.phik_matrix(interval_cols=interval_cols)
phik_overview_df.sort_values(by="churn", inplace=True, axis=0)
phik_overview_df.sort_values(by="churn", inplace=True, axis=1)
sorted_phik_df = churn_df[ phik_overview_df.index ]
phik_overview_df = sorted_phik_df.phik_matrix(interval_cols=interval_cols)
display(phik_overview_df);



In [ ]:


plot_correlation_matrix(
    phik_overview_df.values
,   x_labels          = phik_overview_df.columns
,   y_labels          = phik_overview_df.index
,   vmin              = 0
,   vmax              = 1
,   color_map         = 'RdBu_r'
,   title             = r'Correlação $\phi_K$'
,   figsize=(25,25)
);
plt.tight_layout();


O coeficiente de correlação Phi K consegue calcular as correlações entre variáveis categóricas e numéricas analisando o efeito que as variáveis categóricas possuem sobre as variáveis numéricas intervalares.

Diferente de outros algoritmos de para cálculo de correlação, ele consegue capturar de forma adequada relacionamentos não lineares entre variáveis, mas não consegue indicar a direção do relacionamento, apenas se ele existe e sua intensidade, pois os dados retornados variam entre um intervalo entre 0 e 1.

Por ser um algoritmo ainda novo, a interpretação da intensidade dos relacionamentos pode ser um tanto dificultada. Portanto, as linhas e colunas do dataframe contendo as correlações foram ordenados com base na intensidade do relacionamento com a variável target **churn**. Nesse sentido, as constatações anteriormente realizadas via análise das tabulações cruzadas com a variável target são reforçadas/confirmadas.

Ignorando a primeira coluna/linha (por tratar-se da própria variável target após a ordenação citada acima), podemos verificar que as variáveis que possuem a maior correlação com a mesma são:

- **tenure** indicando que os clientes mais novos possuem maior rotatividade e que, com o passar do tempo, tendem a se fidelizar, conforme análise anterior.

- **monthly_charges** indicando que os clientes que contratam os serviços mais caros (internet de fibra ótica) tendem a ter a maior rotatividade e que clientes que possuem apenas planos telefônicos, tendem a apresentar menor rotatividade (conforme análise anterior).

- **payment_method** indicando que os clientes que realizam pagamento via cheques eletrônicos(depósitos diretos) são os que apresentam a maior rotatividade(conforme análise anterior), possivelmente ligado a facilidade na qual os pagamentos podem ser sustados sem necessidade de contato com a operadora.

Existe uma alta correlação entre os diferentes serviços oferecidos e uma correlação dos mesmos com os valores mensais cobrados. Essa relação possivelmente pode ser explicada sobre o efeito que a contratação desses serviços possuem sobre o valor da mensalidade. Nesse sentido, é difícil interpretar as correlações reportadas por falta de uma intuição adequada sobre o seu significado, efeitos e características.

Mais detalhes sobre o coeficiente Phi K podem ser encontrados no link abaixo, inclusive o artigo no qual o mesmo foi apresentado.

https://towardsdatascience.com/phik-k-get-familiar-with-the-latest-correlation-coefficient-9ba0032b37e7

## 4. Pré-Processamento de dados

Nesta etapa, poderíamos realizar diversas operações de preparação de dados, como por exemplo, tratamento de valores missings (faltantes), limpeza de dados, transformações como one-hot-encoding, seleção de características (feature selection), entre outras não mostradas neste notebook. Lembre-se de não criar uma versão padronizada/normalizada dos dados neste momento (apesar de serem operações de pré-processamento) para evitar o Data Leakage.

### 4.1. Tratamento das variáveis categóricas

Dado o elevado número de variáveis categóricas presentes no data set, iremos adotar a estratégia de **Dummy Encoding** para tranformar os dados categóricos em numéricos e posteriormente removendo as colunas redundantes.


In [ ]:
# transformando variáveis booleanas em numéricas (dummy encoding não é necessário)
for feature in BOOLEAN_FEATURES:
  churn_df[feature] = churn_df[feature].map(BOOLEAN_MAP)

# realizando o dummy encoding usando pandas
dummy_df = pd.get_dummies(
    data        = churn_df[CATEGORICAL_FEATURES]
,   prefix      = CATEGORICAL_FEATURES
,   prefix_sep  = "="
)

# concatenando as variáveis boleanas, categóricas codificads, numéricas e variável target num novo dataset
churn_df = pd.concat([
    churn_df[ BOOLEAN_FEATURES ]
,   dummy_df
,   churn_df[ NUMERICAL_FEATURES ]
,   churn_df[ TARGET_VARIABLE ]
], axis=1)

# removendo variáveis equivalentes internet_service=No = 1
del churn_df["device_protection=No internet service"]
del churn_df["streaming_tv=No internet service"]
del churn_df["tech_support=No internet service"]
del churn_df["online_backup=No internet service"]
del churn_df["streaming_movies=No internet service"]
del churn_df["online_security=No internet service"]

# removendo variáveis equivalentes phone_service=0
del churn_df["multiple_lines=No phone service"]

# removendo variáveis codificada tornadas redundantes pelas deleções acima
del churn_df["multiple_lines=No"]
del churn_df["online_security=No"]
del churn_df["online_backup=No"]
del churn_df["device_protection=No"]
del churn_df["tech_support=No"]
del churn_df["streaming_tv=No"]
del churn_df["streaming_movies=No"]
del churn_df["internet_service=No"]


As colunas resultantes foram renomeadas para melhorar a legibilidade do dataframe.

In [ ]:
new_column_names = {
    'multiple_lines=Yes'                       : 'multiple_lines'
,   'internet_service=DSL'                     : 'dsl'
,   'internet_service=Fiber optic'             : 'fiber_optic'
,   'online_security=Yes'                      : 'online_security'
,   'online_backup=Yes'                        : 'online_backup'
,   'device_protection=Yes'                    : 'device_protection'
,   'tech_support=Yes'                         : 'tech_support'
,   'streaming_tv=Yes'                         : 'streaming_tv'
,   'streaming_movies=Yes'                     : 'streaming_movies'
,   'contract=Month-to-month'                  : 'monthly_contract'
,   'contract=One year'                        : 'one_year_contract'
,   'contract=Two year'                        : 'two_year_contract'
,   'payment_method=Bank transfer (automatic)' : 'bank_transfer'
,   'payment_method=Credit card (automatic)'   : 'credit_card'
,   'payment_method=Electronic check'          : 'electronic_check'
,   'payment_method=Mailed check'              : 'mailed_check'
}
churn_df.rename(columns=new_column_names, inplace=True)

## 4.2 Feature Engineering

Foi verificado anteriormente que as variáveis **tenure**, **monthly_charges** e **payment_method** possuiam correlação significativa com a variável target.

As três variáveis serão combinadas para tentar criar variáveis com melhor poder explicativo.

In [ ]:
orig_df = churn_df.copy()

In [ ]:

churn_df = orig_df.copy()

rows, cols = churn_df.shape
# client factor
# A análise das tabulações cruzadas revelou que a existência de parceiro e dependentes tendem a fidelizar o cliente.
# Em contrapartida, observou-se que clientes na terceira idade proporcionalmente tendem a cancelar os serviços
# de maneira mais frequente.
# A expectativa é de que quanto maior for o client_factor, maior a probabilidade de que ele venha a cancelar o seu contrato
noise_term = np.random.normal(loc=0.0, scale=0.1, size=rows)
churn_df["client_factor"] = (
    np.exp(churn_df["senior_citizen"]) # senior_citizen=1 piora as p
+   np.exp(np.abs(1-churn_df["partner"]))
+   np.exp(np.abs(1-churn_df["dependents"]))
) / 3.0 + noise_term

# internet factor
noise_term = np.random.normal(loc=0.0, scale=0.1, size=rows)
churn_df["internet_factor"] = (
    np.exp(np.abs(1-churn_df["tech_support"])) 
+   np.exp(np.abs(1-churn_df["online_security"])) 
+   np.exp(churn_df["dsl"]) 
+   np.exp(churn_df["fiber_optic"])
) / 4.0 + noise_term

# financial factor
noise_term = np.random.normal(loc=0.0, scale=0.1, size=rows)
churn_df["financial_factor"] = (
    np.exp(churn_df["monthly_contract"]) 
+   np.exp(churn_df["electronic_check"])
+   np.exp(churn_df["paperless_billing"])
) / 3.0 + noise_term

noise_term = np.random.normal(loc=0.0, scale=0.1, size=rows)
churn_df["multi_factor"] = (
    np.exp(churn_df["senior_citizen"]) # senior_citizen=1 piora as p
+   np.exp(np.abs(1-churn_df["partner"]))
+   np.exp(np.abs(1-churn_df["dependents"]))
+   np.exp(np.abs(1-churn_df["tech_support"])) 
+   np.exp(np.abs(1-churn_df["online_security"])) 
+   np.exp(churn_df["dsl"]) 
+   np.exp(churn_df["fiber_optic"])
+   np.exp(churn_df["monthly_contract"]) 
+   np.exp(churn_df["electronic_check"])
+   np.exp(churn_df["paperless_billing"])
) / 10.0 + noise_term

churn_df["churn"] = churn_df.pop("churn")
churn_df.describe()

In [ ]:
scatter_vars = [
  "client_factor", 
  "internet_factor", 
  "financial_factor", 
  "multi_factor", 
  "tenure",
  "monthly_charges",
  "total_charges"
]

sns.pairplot(
    churn_df
,   diag_kind = "hist"    
,   vars      = scatter_vars
,   hue       = "churn"
)


### 4.3. Separação em conjunto de treino e conjunto de teste

Para evitar que os processo de treino do modelo resulte em um modelo sobreajustado aos dados (overfitting) é importante testar o modelo com dados ainda não vistos. Modelos sobreajustados tendem a ter uma performance pior estes dados, sendo isso um forte indicativo de que o modelo não foi capaz de generalizar seu poder preditivo.

É uma boa prática usar um conjunto de teste, uma amostra dos dados que não será usada para a construção do modelo, mas somente no fim do projeto para confirmar a precisão do modelo final.

Trata-se de um boa prática usada para avaliação sistemática da performance do modelo.

Usaremos 80% do conjunto de dados para modelagem e treino, e guardaremos 20% para teste, usando a estratégia train-test-split.

## 4.3 Normalização

Dado as significativas diferenças entre as escalas das variáveis, a ausência de outliers presentes no conjunto de dados e a presença de variáveis numéricas geradas a partir de dados categóricos usando dummy encoding(representadas como 0's ou 1's), o conjunto de dados será tratado usando o procedimento conhecido como normalização. A normalização irá sempre mapear o intervalo de valores observados nas variávéis para o intervalo entre 0 e 1.

A normalização feita abaixo precisará ser posteriormente refeita no pipeline, pois a separação do conjunto de dados de treino e teste ainda não foi realizada, o que poderia ser caracterizado como um data leakage da variável target.

In [ ]:
all_but_target = churn_df.columns.difference([TARGET_VARIABLE])
X = churn_df[all_but_target].values
y = churn_df[TARGET_VARIABLE].values

X_train, X_test, y_train, y_test = train_test_split(
    X
,   y
,   test_size     = TEST_PCT_SIZE
,   shuffle       = True
,   random_state  = RANDOM_STATE
,   stratify      = y # com estratificação
)

X_train_df = pd.DataFrame(X_train, columns=all_but_target)
X_test_df = pd.DataFrame(X_test, columns=all_but_target)
y_train_df = pd.DataFrame(y_train, columns=[TARGET_VARIABLE])
y_test_df = pd.DataFrame(y_test, columns=[TARGET_VARIABLE])

## 5. Modelos de Classificação

### 5.1. Criação e avaliação de modelos: linha base

Não sabemos de antemão quais modelos performarão bem neste conjunto de dados. Assim, usaremos a validação cruzada 10-fold (já detalhada anteriormente) e avaliaremos os modelos usando a métrica de acurácia. Vamos inicialmente configurar os parâmetros de número de folds e métrica de avaliação.

In [ ]:
# Parâmetros e partições da validação cruzada estratificada
kfold = StratifiedKFold(
    n_splits      = K_FOLDS
,   shuffle       = True
,   random_state  = RANDOM_STATE
) 
# configura pandas para exibição de apenas duas casas decimais nas variaveis
pd.set_option('display.float_format', lambda x: '%0.3f' % x)


In [ ]:
# Configuração do pipeline

# Os transformadores numéricos são utilizado spara processamento de todas as variáveis não categóricas.
numeric_transformer = Pipeline([
  ("scaler", StandardScaler())    
])

column_transformer = ColumnTransformer(
  transformers = [
    ("num", numeric_transformer, NUMERICAL_FEATURES_AFTER_FEAT_ENG)
  ],
  # importante usar passthrough quando nem todos os atributos forem processados
  remainder="passthrough" 
)

# Este pipelie será ajustado diversas vezes durante o processo de otimização dos hiper parâmetros.
pipeline = Pipeline([
    # a primeira fase consiste no pré-processamento das variáveis numéricas
    ("feature_scaling", column_transformer),
    # redução de dimesionalidade
    ("reduce_dim", PCA()),
    # O algoritmo de regressão e seus parâmetros serão configurados via gridsearch
    ("classifier", SVC())
])

stk_pipeline = Pipeline([
    # O algoritmo de regressão e seus parâmetros serão configurados via gridsearch
    ("classifier", SVC())                         
])

In [ ]:
def search_hyperparameters(param_grid, num_iter, algo):
  model_settings = MODEL_SETTINGS[algo]
  if model_settings["train"]:
    grid = RandomizedSearchCV(
        estimator           = pipeline
    ,   param_distributions = param_grid
    ,   scoring             = SCORING_METRIC
    ,   cv                  = kfold
    ,   n_iter              = num_iter
    ,   return_train_score  = RETURN_TRAIN_SCORE
    )
    grid.fit(X_train_df, y_train_df)
    save_model(grid, model_settings["model_file"])
  else:
    grid = read_model(model_settings["model_file"])
  print(f"Melhor ROCAUC: {grid.best_score_}")
  print("Melhor estimador -> ")
  pprint(grid.best_params_)
  results_df = pd.DataFrame(grid.cv_results_)
  results_df.to_excel(OUTPUT_TRAINING_FILE_TMPLT.replace("{algo}", algo))
  return grid, results_df

def plot_mean_std(results_df, param_name, new_name):
  results_df[new_name] = results_df[param_name]
  return sns.scatterplot(
      data=results_df
  ,   x="mean_test_score"
  ,   y="std_test_score"
  ,   hue=new_name
  )
  
def show_top_n(results_df, n, column_mapping):
  results_df.sort_values("mean_test_score", inplace=True, ascending=False)
  columns = {new_name: results_df[col_name] for new_name, col_name in column_mapping.items()}
  params_df = pd.DataFrame(columns)
  display(params_df.head(n))
  return params_df

def report_results(grid):
  y_train_hat         = grid.best_estimator_.predict(X_train_df)
  y_test_hat          = grid.best_estimator_.predict(X_test_df)
  
  y_train_prob        = grid.best_estimator_.predict_proba(X_train_df)[:,1]
  y_test_prob         = grid.best_estimator_.predict_proba(X_test_df)[:,1]
  
  acc_train_score     = accuracy_score(y_train, y_train_hat)
  acc_test_score      = accuracy_score(y_test, y_test_hat)  
  
  f1_train_score      = f1_score(y_train, y_train_hat)
  f1_test_score       = f1_score(y_test, y_test_hat)
  
  rocauc_train_score  = roc_auc_score(y_train, y_train_prob)
  rocauc_test_score   = roc_auc_score(y_test, y_test_prob)
  
  print(f"== Test ACC Score     :  {acc_test_score} ===")
  print(f"== Test F1 Score      :  {f1_test_score} ===")
  print(f"== Test ROCAUC Score  :  {rocauc_test_score} ===")
  
  plot_confusion_matrix(
      grid.best_estimator_
  ,   X_test_df
  ,   y_test_df
  )
  plt.show()
  plot_roc_curve(
      grid.best_estimator_
  ,   X_test_df
  ,   y_test_df
  )
  plt.show()
  print(classification_report(y_test, y_test_hat))

def save_model(grid, file_name):
  import pickle
  data = {
    "best_score_"     : grid.best_score_
  , "best_params_"    : grid.best_params_
  , "best_estimator_" : grid.best_estimator_
  , "cv_results_"     : grid.cv_results_
  , "grid"            : grid
  }  
  with open(file_name, "wb") as fh:
    pickle.dump(data, fh)

def read_model(file_name):
  import pickle
  with open(file_name, "rb") as fh:
    data                  = pickle.load(fh)
  grid                  = data["grid"]
  grid.best_score_      = data["best_score_"]
  grid.best_params_     = data["best_params_"]
  grid.best_estimator_  = data["best_estimator_"]
  grid.cv_results_      = data["cv_results_"]
  return grid

In [ ]:
# Realiza a busca por hiperparâmetros
param_grid = [
  # Logistic Regression
  {
    "feature_scaling__num__scaler"  : ["passthrough", MinMaxScaler(), StandardScaler()],
    "reduce_dim"                    : ["passthrough", PCA(n_components=3), PCA(n_components=5), PCA(n_components=10)],
    "classifier"                    : [LogisticRegression()],
    "classifier__n_jobs"            : [-1], # all cpus available
    "classifier__penalty"           : ["elasticnet"],
    "classifier__class_weight"      : ["balanced"],
    "classifier__solver"            : ["saga"],
    "classifier__l1_ratio"          : [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
  },
  {
    "feature_scaling__num__scaler"  : ["passthrough", MinMaxScaler(), StandardScaler()],
    "reduce_dim"                    : ["passthrough", PCA(n_components=3), PCA(n_components=5), PCA(n_components=10)],
    "classifier"                    : [LogisticRegression()],
    "classifier__n_jobs"            : [-1], # all cpus available
    "classifier__penalty"           : ["none"],
    "classifier__class_weight"      : ["balanced"],
  }  
]
if QUICK_RUN:
  logreg_grid, logreg_results_df =  search_hyperparameters(param_grid, 20, "logreg")
else:
  logreg_grid, logreg_results_df =  search_hyperparameters(param_grid, 200, "logreg")

In [ ]:
plot_mean_std(logreg_results_df, "param_classifier__penalty", "penalty")

Os melhores resultados obtidos com a regressão logística obtiveram um score F1 perto de 0.632 com um desvio padrão da métrica calculado aproximadamente 0.02.

In [ ]:
column_mapping={
    "mean_score"      : "mean_test_score"
,   "std_score"       : "std_test_score"
,   "l1_ratio"        : "param_classifier__l1_ratio"
,   "penalty"         : "param_classifier__penalty"
,   "feature_scaling" : "param_feature_scaling__num__scaler"
,   "reduce_dim"      : "param_reduce_dim"
}
logreg_params_df = show_top_n(logreg_results_df, 50, column_mapping)

Os melhores resultados obtidos não usaram nenhum tipo de normalização ou padronização dos dados e o uso de redução de dimensionalidade para 5 features via PCA parece ter um efeito positivo na maioria dos casos

In [ ]:
report_results(logreg_grid)

<<Conclusão>>

In [ ]:
# Realiza a busca por hiperparâmetros
param_grid = [
  # KNeighborsClassifier
  {
    "feature_scaling__num__scaler"  : ["passthrough", MinMaxScaler(), StandardScaler()],
    "reduce_dim"                    : ["passthrough", PCA(n_components=3), PCA(n_components=5), PCA(n_components=10)],
    "classifier"                    : [KNeighborsClassifier()],
    "classifier__n_jobs"            : [-1], # all cpus available
    "classifier__algorithm"         : ["kd_tree"],
    "classifier__metric"            : ["minkowski"],
    "classifier__p"                 : [0.5, 1.0, 1.5, 2.0], # manhattan, euclidean
    "classifier__n_neighbors"       : [5, 7, 10, 13, 15, 17, 20],
    "classifier__weights"           : ["uniform", "distance"]
  }  
]
if QUICK_RUN:
  knn_grid, knn_results_df =  search_hyperparameters(param_grid, 20, "knn")
else:
  knn_grid, knn_results_df =  search_hyperparameters(param_grid, 200, "knn")

In [ ]:
plot_mean_std(knn_results_df, "param_classifier__n_neighbors", "n_neighbors")

Os melhores resultados obtidos com o algoritmo KNN obtiveram um score F1 inferior a 0.6 com um desvio padrão da métrica calculado em aproximadamente 0.03.

In [ ]:
column_mapping = {
  "mean_score"      : "mean_test_score"
, "std_score"       : "std_test_score"
, "n_neighbors"     : "param_classifier__n_neighbors"
, "minkowski-p"     : "param_classifier__p"
, "weights"         : "param_classifier__weights"
, "feature_scaling" : "param_feature_scaling__num__scaler"
, "reduce_dim"      : "param_reduce_dim"
}
knn_params_df = show_top_n(knn_results_df, 50, column_mapping)

Os melhores resultados obtidos utilizaram-se de 13 ou mair vizinhos e a redução de dimensionalidade PCA não foi utilizada. Os melhores resultados tipicamente utilizaram o parâmetro p da métrica de Minkowski menor do que 2 na maioria dos casos (indicando que a distância euclidiana não é adequada para uso nesse conjunto de dados).

In [ ]:
report_results(knn_grid)

O cômputo do score F1 usando o conjunto de treinamento indica claramente que ocorreu overfitting na parametrização usada para o algoritmo KNN.

In [ ]:
# Realiza a busca por hiperparâmetros
param_grid = [
  # GaussianNB
  {
    "feature_scaling__num__scaler"  : ["passthrough", MinMaxScaler(), StandardScaler()],
    "reduce_dim"                    : ["passthrough", PCA(n_components=3), PCA(n_components=5), PCA(n_components=10)],
    "classifier"                    : [GaussianNB()],
    "classifier__var_smoothing"     : [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
  }  
]
if QUICK_RUN:
  nb_grid, nb_results_df =  search_hyperparameters(param_grid, 20, "nb")
else:  
  nb_grid, nb_results_df =  search_hyperparameters(param_grid, 200, "nb")

In [ ]:
plot_mean_std(nb_results_df, "param_classifier__var_smoothing", "var_smoothing")

Os melhores resultados obtidos com o algoritmo KNN obtiveram um score F1 superior a 0.62 com um desvio padrão da métrica calculado em aproximadamente 0.018

In [ ]:
column_mapping = {
  "mean_score"      : "mean_test_score"
, "std_score"       : "std_test_score"
, "var_smoothing"   : "param_classifier__var_smoothing"
, "feature_scaling" : "param_feature_scaling__num__scaler"
, "reduce_dim"      : "param_reduce_dim"
}
pd.set_option('display.float_format', lambda x: '%0.10f' % x)
nb_params_df = show_top_n(nb_results_df, 50, column_mapping)
pd.set_option('display.float_format', lambda x: '%0.3f' % x)

O algoritmo de NaiveBayes não possui muitos hiperparâmetros e comporta-se basicamente de maneira indiferente quanto a aplicação ou não do procedimento de normalização ou padronização.

A redução de dimensionalidade, via PCA não ajuda em nada no algoritmo.

O conjunto de dados, quando inicialmente analisado, parecia ser um bom candidato para aplicação deste algoritmo devido a elevada presença de variáveis categóricas, entretanto, a performance observada do mesmo foi inferior à performance da regressão logística, até o momento o melhor algoritmo observado.

In [ ]:
report_results(nb_grid)

In [ ]:
# Realiza a busca por hiperparâmetros
param_grid = [
  # DecisionTreeClassifier
  {
    "feature_scaling__num__scaler"  : ["passthrough", MinMaxScaler(), StandardScaler()],
    "reduce_dim"                    : ["passthrough", PCA(n_components=3), PCA(n_components=5), PCA(n_components=10)],
    "classifier"                    : [DecisionTreeClassifier()],
    "classifier__class_weight"      : [None, "balanced"],
    "classifier__criterion"         : ["gini", "entropy"],
    "classifier__splitter"          : ["best", "random"],
    "classifier__max_features"      : [None, "auto", "sqrt", "log2"],
  }  
]
if QUICK_RUN:
  dt_grid, dt_results_df =  search_hyperparameters(param_grid, 20, "dt")
else:
  dt_grid, dt_results_df =  search_hyperparameters(param_grid, 200, "dt")

In [ ]:
plot_mean_std(dt_results_df, "param_classifier__criterion", "criterion")

Os melhores resultados obtidos com o algoritmo de Árvore de Decisões obtiveram um score F1 com cerca de 0.50 com um desvio padrão da métrica calculado em aproximadamente 0.003

Surpreendentemente, o algoritmo se comporta pior do que a regressão logística.

In [ ]:
column_mapping = {
  "mean_score"      : "mean_test_score"
, "std_score"       : "std_test_score"
, "feature_scaling" : "param_feature_scaling__num__scaler"
, "reduce_dim"      : "param_reduce_dim"
, "class_weight"    : "param_classifier__class_weight"
, "criterion"       : "param_classifier__criterion"
, "splitter"        : "param_classifier__splitter"
, "max_features"    : "param_classifier__max_features"
}
dt_params_df = show_top_n(dt_results_df, 50, column_mapping)

O algoritmo de Árvore de Decisão comportou-se melhor com o critério *gini* e parece ter se beneficiado do procedimento de redução de dimensionalidade usando PCA. O balanceamento de classes parece ter ajudado também no processo de treino.

In [ ]:
report_results(dt_grid)

Assim como para o algoritmo KNN, O cômputo do score F1 usando o conjunto de treinamento indica claramente que ocorreu overfitting na parametrização usada para o algoritmo de Árvore de Decisões.

In [ ]:
# Realiza a busca por hiperparâmetros
param_grid = [
  # SVC
  {
    "feature_scaling__num__scaler"  : [MinMaxScaler(), StandardScaler()], # SVC precisa ter os argumentos escalonados para uma melhor performance
    "reduce_dim"                    : ["passthrough", PCA(n_components=5), PCA(n_components=10)],
    "classifier"                    : [SVC(probability=True)],
    "classifier__kernel"            : ["linear","rbf"],
    "classifier__gamma"             : ["scale", "auto"],
    "classifier__class_weight"      : ["balanced"],
    #"classifier__C"                 : [0.0, 0.5, 1.0, 1.5, 2.0],
  }  
]
if QUICK_RUN:
  svm_grid, svm_results_df =  search_hyperparameters(param_grid, 5, "svm")
else:  
  svm_grid, svm_results_df =  search_hyperparameters(param_grid, 50, "svm")

In [ ]:
plot_mean_std(svm_results_df, "param_classifier__kernel", "kernel")

Surpreendentemente, o SVM com kernel linear se comportou melhor do que o kernel de *radial basis function*, usado por padrão.

Os melhores scores F1 obtidos

In [ ]:
column_mapping = {
  "mean_score"      : "mean_test_score"
, "std_score"       : "std_test_score"
, "feature_scaling" : "param_feature_scaling__num__scaler"
, "reduce_dim"      : "param_reduce_dim"
, "gamma"           : "param_classifier__gamma"
, "kernel"          : "param_classifier__kernel"
}
svm_params_df = show_top_n(svm_results_df, 50, column_mapping)

In [ ]:
report_results(svm_grid)

In [ ]:
# Realiza a busca por hiperparâmetros
param_grid = [
  # AdaBoostClassifier
  {
    "feature_scaling__num__scaler"    : ["passthrough", MinMaxScaler(), StandardScaler()],
    "reduce_dim"                      : ["passthrough", PCA(n_components=5), PCA(n_components=10)],
    "classifier"                      : [AdaBoostClassifier()],
    "classifier__n_estimators"        : [25, 50, 75, 100],
    "classifier__learning_rate"       : [0.001, 0.01, 0.1, 1.0]
  }  
]
if QUICK_RUN:
  ada_grid, ada_results_df =  search_hyperparameters(param_grid, 5, "ada")
else:  
  ada_grid, ada_results_df =  search_hyperparameters(param_grid, 64, "ada")

In [ ]:
column_mapping = {
  "mean_score"      : "mean_test_score"
, "std_score"       : "std_test_score"
, "feature_scaling" : "param_feature_scaling__num__scaler"
, "reduce_dim"      : "param_reduce_dim"
, "n_estimators"    : "param_classifier__n_estimators"
, "learning_rate"   : "param_classifier__learning_rate"
}
ada_params_df = show_top_n(ada_results_df, 50, column_mapping)

In [ ]:
report_results(ada_grid)

In [ ]:
# Realiza a busca por hiperparâmetros
param_grid = [
  # GradientBoostingClassifier
  {
    "feature_scaling__num__scaler"    : ["passthrough", MinMaxScaler(), StandardScaler()],
    "reduce_dim"                      : ["passthrough", PCA(n_components=5), PCA(n_components=10)],
    "classifier"                      : [GradientBoostingClassifier()],
    "classifier__loss"                : ["log_loss", "deviance", "exponential"],
    "classifier__n_estimators"        : [50, 75, 100, 150],
    "classifier__learning_rate"       : [0.1, 0.3, 0.5, 0.7, 1.0],
    "classifier__max_depth"           : [3, 5, 10],
    "classifier__max_features"        : [None, "sqrt", "log2"]
  }  
]
if QUICK_RUN:
  gb_grid, gb_results_df =  search_hyperparameters(param_grid, 10, "gb")
else:  
  gb_grid, gb_results_df =  search_hyperparameters(param_grid, 100, "gb")

In [ ]:
plot_mean_std(gb_results_df, "param_classifier__n_estimators", "n_estimators")

In [ ]:
column_mapping = {
  "mean_score"      : "mean_test_score"
, "std_score"       : "std_test_score"
, "feature_scaling" : "param_feature_scaling__num__scaler"
, "reduce_dim"      : "param_reduce_dim"
, "loss"            : "param_classifier__loss"
, "n_estimators"    : "param_classifier__n_estimators"
, "loss"            : "param_classifier__loss"
, "learning_rate"   : "param_classifier__learning_rate"
, "max_depth"       : "param_classifier__max_depth"
, "max_features"    : "param_classifier__max_features"
}
gb_params_df = show_top_n(gb_results_df, 50, column_mapping)

In [ ]:
report_results(gb_grid)

In [ ]:
# Realiza a busca por hiperparâmetros
param_grid = [
  # RandomForestClassifier
  {
    "feature_scaling__num__scaler"    : ["passthrough", MinMaxScaler(), StandardScaler()],
    "reduce_dim"                      : ["passthrough"], #PCA(n_components=5), PCA(n_components=10)],
    "classifier"                      : [RandomForestClassifier()],
    "classifier__n_estimators"        : [50, 100, 150],
    "classifier__criterion"           : ["gini", "entropy"],
    "classifier__bootstrap"           : [True, False],
    "classifier__n_jobs"              : [-1],
    "classifier__class_weight"        : ["balanced", "balanced_subsample"],
  }
]
if QUICK_RUN:
  rf_grid, rf_results_df =  search_hyperparameters(param_grid, 10, "rf")
else:
  rf_grid, rf_results_df =  search_hyperparameters(param_grid, 100, "rf")

In [ ]:
plot_mean_std(rf_results_df, "param_classifier__n_estimators", "n_estimators")

In [ ]:
column_mapping = {
  "mean_score"      : "mean_test_score"
, "std_score"       : "std_test_score"
, "feature_scaling" : "param_feature_scaling__num__scaler"
, "reduce_dim"      : "param_reduce_dim"
, "n_estimators"    : "param_classifier__n_estimators"
, "criterion"       : "param_classifier__criterion"
, "bootstrap"       : "param_classifier__bootstrap"
, "class_weight"    : "param_classifier__class_weight"
}
rf_params_df = show_top_n(rf_results_df, 50, column_mapping)

In [ ]:
report_results(rf_grid)

In [ ]:
from itertools import combinations

model_settings = MODEL_SETTINGS["vt"]

if not model_settings["train"]:
    with open(model_settings["model_file"], "rb") as fh:
      classifier = pickle.load(fh)
else:
  def compute_score(estimator):
    y_train_prob        = grid.best_estimator_.predict_proba(X_train_df)[:,1]
    rocauc_train_score  = roc_auc_score(y_train, y_train_prob)
    return rocauc_train_score

  # Realiza a busca por hiperparâmetros
  estimators_weights = [
      ( logreg_grid.best_estimator_,  logreg_grid.best_score_ )
  ,   ( knn_grid.best_estimator_,     knn_grid.best_score_    )
  ,   ( nb_grid.best_estimator_,      nb_grid.best_score_     )
  ,   ( svm_grid.best_estimator_,     svm_grid.best_score_    )
  ,   ( ada_grid.best_estimator_,     ada_grid.best_score_    )
  ,   ( gb_grid.best_estimator_,      gb_grid.best_score_     )
  ,   ( rf_grid.best_estimator_,      rf_grid.best_score_     )
  ]

  best_score      = -99999
  best_estimators = None
  best_type       = None

  for voting_type in ["hard"]: # não estou usando "soft" voting para permitir a melhor contribuição de vários modelos
    for i in [3, 5, 7]:
      for comb_estimators_weights in combinations(estimators_weights, i):
        # https://stackoverflow.com/questions/13635032/what-is-the-inverse-function-of-zip-in-python
        comb_estimators, weights = zip(*comb_estimators_weights)
        classifier = EnsembleVoteClassifier(clfs=comb_estimators, weights=weights, voting=voting_type, refit=False)  
        classifier.fit(None, y_train_df)
        score = compute_score(classifier)
        if score > best_score:
          best_score      = score
          best_estimators = comb_estimators_weights
          best_type       = voting_type

  comb_estimators, weights = zip(*best_estimators)
  classifier = EnsembleVoteClassifier(clfs=comb_estimators, weights=weights, voting=best_type, refit=False)    
  classifier.fit(None, y_train_df)
  with open(model_settings["model_file"], "wb") as fh:
    pickle.dump(classifier, fh)
  
print(f"best combination of estimators: {classifier.clfs}")
print(f"estimators weights: {classifier.weights}")
print(f"voting type: {classifier.voting}")

class PseudoGrid:
  def __init__(self, estimator):
      self.best_estimator_ = estimator
grid = PseudoGrid(classifier)
report_results(grid)

<<CONCLUSÃO>>

In [ ]:
# Lista que armazenará os modelos
models = []

# Criando os modelos e adicionando-os na lista de modelos
models.append(('LR', LogisticRegression(max_iter=200))) 
models.append(('KNN', KNeighborsClassifier())) 
models.append(('CART', DecisionTreeClassifier())) 
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

Vamos adicionar também os algoritmos de ensemble que estudamos:

In [ ]:
np.random.seed(7) # definindo uma semente global

# definindo os parâmetros do classificador base para o BaggingClassifier
base = DecisionTreeClassifier()
num_trees = 100
max_features = 3

# criando os modelos para o VotingClassifier - TODO: você poderia experimentar outras variações aqui!
bases = []
model1 = LogisticRegression(max_iter=200)
bases.append(('logistic', model1))
model2 = DecisionTreeClassifier()
bases.append(('cart', model2))
model3 = SVC()
bases.append(('svm', model3))

# Criando os modelos e adicionando-os na lista de modelos
models.append(('Bagging', BaggingClassifier(base_estimator=base, n_estimators=num_trees)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('Ada', AdaBoostClassifier(n_estimators=num_trees)))
models.append(('GB', GradientBoostingClassifier(n_estimators=num_trees)))
models.append(('Voting', VotingClassifier(bases)))

Agora vamos comparar os resultados modelos criados, treinando-os com os dados do conjunto de treino e utilizando a técnica de validação cruzada. Para cada um dos modelos criados, executaremos a validação cruzada e, em seguida, exibiremos a acurácia média e o desvio padrão de cada um. Faremos isso tanto para o dataset original quanto para o dataset sem missings.

In [ ]:
# Aqui iremos armazenar os resultados tanto para o dataset original quanto para o dataset sem missings
results = []
names = []

In [ ]:
np.random.seed(7) # definindo uma semente global

# Avaliação dos modelos - dataset original

for name, model in models:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
np.random.seed(7) # definindo uma semente global

# Avaliação dos modelos - dataset sem missings

for name, model in models:
    cv_results = cross_val_score(model, X_train_sm, y_train_sm, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

*Dica: organize os resultados numéricos em tabelas, para facilitar a sua comparação.*

Estes resultados sugerem que, para ambos os datasets, diversos modelos têm potencial de trazerem bons resultados, porém, vale observar que estes são apenas valores médios de acurácia, sendo também prudente também observar a distribuição dos resultados dos folds da validação cruzada. Faremos isto comparando os modelos usando boxplots. Os 11 primeiros boxplots são referentes ao dataset original e os seguintes, ao dataset com tratamento de missings.

In [ ]:
# Comparação dos modelos
fig = plt.figure(figsize=(15,10)) 
fig.suptitle('Comparação dos Modelos - Dataset original e com tratamento de missings') 
ax = fig.add_subplot(111) 
plt.boxplot(results) 
ax.set_xticklabels(names) 
plt.show()

Os resultados mostram algumas diferenças comparando o dataset original e o dataset com tratamento de missings, parecendo, inicialmente, que o dataset com tratamento de missings gerou melhores resultados (especialmente no KNN: observe a distribuição dos valores). Já considerando a mediana da acurácia marcada como a linha laranja do boxplot, os modelos de regressão logística parecem ter alcançado os melhores resultados.

A seguir, repetiremos este processo usando uma visão padronizada e outra normalizada do conjunto de dados de treinamento.

### 5.2. Criação e avaliação de modelos: dados padronizados e normalizados

Como suspeitamos que as diferentes distribuições dos dados brutos possam impactar negativamente a habilidade de alguns modelos, vamos agora experimentar as visões do dataset padronizado e normalizado, comparando com a visão original do dataset, com e tratamento de missings. Na padronização (*StandardScaler*), os dados serão transformados de modo que cada atributo tenha média 0 e um desvio padrão 1; na normalização (*MinMaxScaler*), cada atributo é redimensionado para um novo intervalo entre 0 e 1.

Para evitar o vazamento de dados (*data leakage*) nestas transformações, vamos usar pipelines que padronizam os dados e constroem o modelo para cada fold de teste de validação cruzada. Dessa forma, podemos obter uma estimativa justa de como cada modelo com dados padronizados pode funcionar com dados não vistos.

*OBS: Repare que neste notebook estamos usando um código mais "limpo" do que nos anteriores.*

In [ ]:
np.random.seed(7) # definindo uma semente global

# Aqui iremos armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []


# Criando os elementos do pipeline

# Algoritmos que serão utilizados
reg_log = ('LR', LogisticRegression(max_iter=200))
knn = ('KNN', KNeighborsClassifier())
cart = ('CART', DecisionTreeClassifier())
naive_bayes = ('NB', GaussianNB())
svm = ('SVM', SVC())
bagging = ('Bag', BaggingClassifier(base_estimator=base, n_estimators=num_trees))
random_forest = ('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features))
extra_trees = ('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features))
adaboost = ('Ada', AdaBoostClassifier(n_estimators=num_trees))
gradient_boosting = ('GB', GradientBoostingClassifier(n_estimators=num_trees))
voting = ('Voting', VotingClassifier(bases))

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())


# Montando os pipelines

# Dataset original
pipelines.append(('LR-orig', Pipeline([reg_log]))) 
pipelines.append(('KNN-orig', Pipeline([knn])))
pipelines.append(('CART-orig', Pipeline([cart])))
pipelines.append(('NB-orig', Pipeline([naive_bayes])))
pipelines.append(('SVM-orig', Pipeline([svm])))
pipelines.append(('Bag-orig', Pipeline([bagging])))
pipelines.append(('RF-orig', Pipeline([random_forest])))
pipelines.append(('ET-orig', Pipeline([extra_trees])))
pipelines.append(('Ada-orig', Pipeline([adaboost])))
pipelines.append(('GB-orig', Pipeline([gradient_boosting])))
pipelines.append(('Vot-orig', Pipeline([voting])))

# Padronização do dataset original
pipelines.append(('LR-padr', Pipeline([standard_scaler, reg_log]))) 
pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))
pipelines.append(('Bag-padr', Pipeline([standard_scaler, bagging]))) 
pipelines.append(('RF-padr', Pipeline([standard_scaler, random_forest])))
pipelines.append(('ET-padr', Pipeline([standard_scaler, extra_trees])))
pipelines.append(('Ada-padr', Pipeline([standard_scaler, adaboost])))
pipelines.append(('GB-padr', Pipeline([standard_scaler, gradient_boosting])))
pipelines.append(('Vot-padr', Pipeline([standard_scaler, voting])))

# Normalização do dataset original
pipelines.append(('LR-norm', Pipeline([min_max_scaler, reg_log]))) 
pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))
pipelines.append(('Bag-norm', Pipeline([min_max_scaler, bagging]))) 
pipelines.append(('RF-norm', Pipeline([min_max_scaler, random_forest])))
pipelines.append(('ET-norm', Pipeline([min_max_scaler, extra_trees])))
pipelines.append(('Ada-norm', Pipeline([min_max_scaler, adaboost])))
pipelines.append(('GB-norm', Pipeline([min_max_scaler, gradient_boosting])))
pipelines.append(('Vot-norm', Pipeline([min_max_scaler, voting])))

# Executando os pipelines - datasets sem tratamento de missings
print("-- Datasets SEM tratamento de missings")
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 3 casas decimais
    print(msg)

# Executando os pipelines - datasets com tratamento de missings
print("-- Datasets COM tratamento de missings")
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train_sm, y_train_sm, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 3 casas decimais
    print(msg)

Vamos analisar estes resultados graficamente:

*OBS: você pode preferir fazer um experimento com menos variações para comparar melhor os resultados graficamente, ou mesmo incluir neste gráfico algumas linhas verticais para separar as diferentes visões do dataset.*

In [ ]:
# Comparação dos modelos
fig = plt.figure(figsize=(25,6))
fig.suptitle('Comparação dos modelos - Dataset orginal, padronizado e normalizado, com e sem tratamento de missings') 
ax = fig.add_subplot(111) 
plt.boxplot(results) 
ax.set_xticklabels(names, rotation=90)
plt.show()

Neste primeiro experimento, rodamos 66 configurações: 11 diferentes algoritmos e 6 diferentes visões do nosso dataset!

Para o dataset **Sem tratamento de missings**, os melhores modelos em termos de acurácia foram: ET-orig (0,779), Bag-orig (0,77), LR-padr	(0,77), GB-orig	(0,769) e Bag-padr (0,767). Já para o dataset **Com tratamento de missings**, os melhores modelos foram: Vot-orig (0,774), ET-norm (0,77), SVM-norm (0,766), LR-padr (0,764) e RF-norm (0,764).

Vamos agora fazer um **novo experimento**, fazendo o ajuste do SVM e do KNN, variando os seus hiperparâmetros a fim de buscar configurações que possam gerar resultados melhores.

*OBS: Você poderia se aprofundar em outros algoritmos também.*

### 5.3. Ajuste dos Modelos (pipeline + gridsearch)

#### Ajuste do KNN

Vamos começar ajustando parâmetros como o número de vizinhos e as métricas de distância para o KNN. Para tal, tentaremos todos os valores ímpares de k entre 1 a 21 e as métricas de distância euclidiana, manhattan e minkowski. Usando o pipeline, cada valor de k e de distância será avaliado usando a validação cruzada 10-fold no conjunto de dados sem tratamento de missings e com as visões padronizada e normalizada, que mostrou melhores resultados do que os dados originais.

In [ ]:
# Tuning do KNN

# Baseado em https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html

np.random.seed(7) # definindo uma semente global

pipelines = []

# definindo os componentes do pipeline
knn = ('KNN', KNeighborsClassifier())
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

pipelines.append(('knn-orig', Pipeline(steps=[knn]))) # OBS: "steps=" é opcional
pipelines.append(('knn-padr', Pipeline(steps=[standard_scaler, knn])))
pipelines.append(('knn-norm', Pipeline(steps=[min_max_scaler, knn])))

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'KNN__n_neighbors': [1,3,5,7,9,11,13,15,17,19,21],
    'KNN__metric': ["euclidean", "manhattan", "minkowski"],
}

# Dataset sem tratamento de missings
for name, model in pipelines:    
    # prepara e executa o GridSearchCV
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid.fit(X_train, y_train)

    # imprime a melhor configuração
    print("Sem tratamento de missings: %s - Melhor: %f usando %s" % (name, grid.best_score_, grid.best_params_)) 

    # imprime todas as configurações
    #means = grid.cv_results_['mean_test_score']
    #stds = grid.cv_results_['std_test_score']
    #params = grid.cv_results_['params']
    #for mean, stdev, param in zip(means, stds, params):
        #print("%f (%f): %r" % (mean, stdev, param))

# Dataset com tratamento de missings
for name, model in pipelines:    
    # prepara e executa o GridSearchCV
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid.fit(X_train_sm, y_train_sm)

    # imprime a melhor configuração
    print("Com tratamento de missings: %s - Melhor: %f usando %s" % (name, grid.best_score_, grid.best_params_)) 

Os resultados mostram que a melhor configuração encontrada utiliza o dataset com tratamento de missings, com dados padronizados, distância de manhattan e k = 15.

#### Ajuste do SVM
Iremos ajustar dois dos principais hiperparâmetros do algoritmo SVM: o valor de C (o quanto flexibilizar a margem) e o tipo de kernel utilizado. No Scikit-Learn, o padrão para o algoritmo SVM (implementado pela classe SVC) é usar o kernel da Função Base Radial (RBF) e o valor C definido como 1.0. 

Iremos testar outros valores para estes hiperparâmetros, e cada combinação de valores será avaliada usando a função GridSearchCV, como fizemos anteriormente para o KNN.

In [ ]:
# Tuning do SVM - DEMORA MUITO ESTE BLOCO DE CÓDIGO

# Baseado em https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html

np.random.seed(7) # definindo uma semente global

pipelines = []

# definindo os componentes do pipeline
svm = ('SVM', SVC())
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

pipelines.append(('svm-orig', Pipeline(steps=[svm]))) # OBS: "steps=" é opcional
pipelines.append(('svm-padr', Pipeline(steps=[standard_scaler, svm])))
pipelines.append(('svm-norm', Pipeline(steps=[min_max_scaler, svm])))

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'SVM__C': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0],
    'SVM__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

# Dataset sem tratamento de missings
for name, model in pipelines:    
    # prepara e executa o GridSearchCV
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid.fit(X_train, y_train)

    # imprime a melhor configuração
    print("Sem tratamento de missings: %s - Melhor: %f usando %s" % (name, grid.best_score_, grid.best_params_)) 

    # imprime todas as configurações
    #means = grid.cv_results_['mean_test_score']
    #stds = grid.cv_results_['std_test_score']
    #params = grid.cv_results_['params']
    #for mean, stdev, param in zip(means, stds, params):
        #print("%f (%f): %r" % (mean, stdev, param))

# Dataset com tratamento de missings
for name, model in pipelines:    
    # prepara e executa o GridSearchCV
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid.fit(X_train_sm, y_train_sm)

    # imprime a melhor configuração
    print("Com tratamento de missings: %s - Melhor: %f usando %s" % (name, grid.best_score_, grid.best_params_)) 

Podemos ver que mesmo a configuração do SVM que alcançou a maior acurácia não supera a acurácia mais alta que conseguimos até o momento, com ensembles.


**Exercício:** Experimente variar os hiperparâmetros de outros algoritmos para verificar se é possível encontrar uma configuração de modelo que supere os melhoresd resultados até o momento.

## 7. Finalização do Modelo

Analisando os resultados até aqui, verificamos que o modelo que mostrou melhor acurácia média para o problema foi o que usou Extra Trees como algoritmo (apesar de ter um  desvio padrão relativamente alto). Relembrando o Experimento 1 (uma vez que o Experimento 2 não trouxe resultados melhores), nossos resultados foram:

*Para o dataset **Sem tratamento de missings**, os melhores modelos em termos de acurácia foram: ET-orig (0,779), Bag-orig (0,77), LR-padr	(0,77), GB-orig	(0,769) e Bag-padr (0,767). Já para o dataset **Com tratamento de missings**, os melhores modelos foram: Vot-orig (0,774), ET-norm (0,77), SVM-norm (0,766), LR-padr (0,764) e RF-norm (0,764).*

Examinando também o desvio padrão, poderíamos, por exemplo, optar por utilizar o modelo construído com o algoritmo de Regressão Logística, com os dados sem tratamento de missings, visão padronizada. Considerando o dataset "Sem tratamento de missings", este modelo ficou na 2a posição em termos de acurácia média, mas com um desvio padrão menor do que o que alcançou a 1a posição. Além disso, explicar como funciona este modelo para os usuários não técnicos tende a ser mais simples.

A seguir, finalizaremos este modelo, treinando-o em todo o conjunto de dados de treinamento (sem validação cruzada) e faremos predições para o conjunto de dados de teste que foi separado logo no início do exemplo, a fim de confirmarmos nossas descobertas.

Primeiro, iremos realizar a padronização dos dados de entrada. Depois, treinaremos o modelo e exibiremos a acurácia de teste, a matriz de confusão e o relatório de classificação.

In [ ]:
# Preparação do modelo
scaler = StandardScaler().fit(X_train) # ajuste do scaler com o conjunto de treino
rescaledX = scaler.transform(X_train) # aplicação da padronização no conjunto de treino
model = LogisticRegression(max_iter=200) # substitua aqui se quiser usar outro modelo
model.fit(rescaledX, y_train)

# Estimativa da acurácia no conjunto de teste
rescaledTestX = scaler.transform(X_test) # aplicação da padronização no conjunto de teste
predictions = model.predict(rescaledTestX)
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

Por meio do conjunto de teste, verificamos que alcançamos uma acurácia de 77,22% em dados não vistos. Este resultado foi ainda melhor do que a nossa avaliação anterior da regressão logíistica. Valores semelhantes são esperados quando este modelo estiver executando em produção e fazendo predições para novos dados.

Vamos agora preparar o modelo para utilização. Para isso, vamos treiná-lo com todo o dataset, e não apenas o conjunto de treino.

In [ ]:
# Preparação do modelo com TODO o dataset (e não apenas a base de treino)
scaler = StandardScaler().fit(X) # ajuste do scaler com TODO o dataset
rescaledX = scaler.transform(X) # aplicação da padronização com TODO o dataset
model.fit(rescaledX, y)

## 8. Aplicando o modelo em dados não vistos

Agora imagine que chegaram 3 novas instâncias, mas não sabemos a classe de saída. Podemos então aplicar nosso modelo recém-treinado para estimar as classes! Para tal, será necessário antes padronizar os dados (usando a mesma escala dos dados usados treinamento do modelo!).

In [ ]:
# Novos dados - não sabemos a classe!
data = {'preg':  [1, 9, 5],
        'plas': [90, 100, 110],
        'pres': [50, 60, 50],
        'skin': [30, 30, 30],
        'test': [100, 100, 100],
        'mass': [20.0, 30.0, 40.0],
        'pedi': [1.0, 2.0, 1.0],
        'age': [15, 40, 40],  
        }

atributos = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age']
entrada = pd.DataFrame(data, columns=atributos)

array_entrada = entrada.values
X_entrada = array_entrada[:,0:8].astype(float)
print(X_entrada)

In [ ]:
# Padronização nos dados de entrada usando o scaler utilizado em X
rescaledEntradaX = scaler.transform(X_entrada)
print(rescaledEntradaX)

In [ ]:
# Estimativa de classes dos dados de entrada
saidas = model.predict(rescaledEntradaX)
print(saidas)

## Resumo

Resumidamente, neste exemplo trabalhamos com um problema de classificação binária de ponta a ponta. As etapas abordadas foram:
* Definição do problema (Pima Indians Diabetes).
* Carga dos dados
* Análise  e tratamento dos dados (verificamos que estavam na mesma escala, mas com diferentes distribuições de dados e possíves missings).
* Avaliação de modelos de linha base, considerando o dataset original e com tratamento de missings.
* Avaliação de modelos com normalização e padronização dos dados.
* Ajuste dos modelos, buscando melhorar o KNN e o SVM.
* Finalização do modelo (use todos os dados de treinamento e valide usando o conjunto de dados de teste).

É importante ressaltar que este exemplo não buscou ser exaustivo, apresentando apenas uma parte dos muitos recursos disponíveis na biblioteca Scikit-Learn. Poderíamos ter testado outras operações de pré-processamento de dados (como feature selection), outros valores de hiperparâmetros e ainda, outros modelos de classificação. Recomendamos que você explore a documentação disponível e incremente este notebook com novas possibilidades.

## Para saber mais:
* Statistical Significance Tests for Comparing Machine Learning Algorithms: https://machinelearningmastery.com/statistical-significance-tests-for-comparing-machine-learning-algorithms/
* Hypothesis Test for Comparing Machine Learning Algorithms: https://machinelearningmastery.com/hypothesis-test-for-comparing-machine-learning-algorithms/
* How to Calculate Parametric Statistical Hypothesis Tests in Python: https://machinelearningmastery.com/parametric-statistical-significance-tests-in-python/
* How to Use Statistical Significance Tests to Interpret Machine Learning Results: https://machinelearningmastery.com/use-statistical-significance-tests-interpret-machine-learning-results/
* 17 Statistical Hypothesis Tests in Python (Cheat Sheet): https://machinelearningmastery.com/statistical-hypothesis-tests-in-python-cheat-sheet/
